# Generating Word Embeddings - Lab

## Introduction

In this lab, you'll learn how to generate word embeddings by training a Word2Vec model, and then embedding layers into deep neural networks for NLP!

## Objectives

You will be able to:

- Train a Word2Vec model and transform words into vectors 
- Obtain most similar words by using methods associated with word vectors 


## Getting Started

In this lab, you'll start by creating your own word embeddings by making use of the Word2Vec model. Then, you'll move onto building neural networks that make use of **_Embedding Layers_** to accomplish the same end-goal, but directly in your model. 

As you've seen, the easiest way to make use of Word2Vec is to import it from the [Gensim Library](https://radimrehurek.com/gensim/). This model contains a full implementation of Word2Vec, which you can use to begin training immediately. For this lab, you'll be working with the [News Category Dataset from Kaggle](https://www.kaggle.com/rmisra/news-category-dataset/version/2#_=_).  This dataset contains headlines and article descriptions from the news, as well as categories for which type of article they belong to.

Run the cell below to import everything you'll need for this lab. 

In [1]:
import pandas as pd
import numpy as np
np.random.seed(0)
from gensim.models import Word2Vec
from nltk import word_tokenize

Now, import the data. The data is stored in the file `'News_Category_Dataset_v2.json'`.  This file is compressed, so that it can be more easily stored in a GitHub repo. **_Make sure to unzip the file before continuing!_**

In the cell below, use the `read_json()` function from Pandas to read the dataset into a DataFrame. Be sure to include the parameter `lines=True` when reading in the dataset!

Once you've imported the data, inspect the first few rows of the DataFrame to see what your data looks like. 

In [2]:
df = pd.read_json("News_Category_Dataset_v2.json", lines=True)
df

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26
...,...,...,...,...,...,...
200848,TECH,RIM CEO Thorsten Heins' 'Significant' Plans Fo...,"Reuters, Reuters",https://www.huffingtonpost.com/entry/rim-ceo-t...,Verizon Wireless and AT&T are already promotin...,2012-01-28
200849,SPORTS,Maria Sharapova Stunned By Victoria Azarenka I...,,https://www.huffingtonpost.com/entry/maria-sha...,"Afterward, Azarenka, more effusive with the pr...",2012-01-28
200850,SPORTS,"Giants Over Patriots, Jets Over Colts Among M...",,https://www.huffingtonpost.com/entry/super-bow...,"Leading up to Super Bowl XLVI, the most talked...",2012-01-28
200851,SPORTS,Aldon Smith Arrested: 49ers Linebacker Busted ...,,https://www.huffingtonpost.com/entry/aldon-smi...,CORRECTION: An earlier version of this story i...,2012-01-28


## Preparing the Data

Since you're working with text data, you need to do some basic preprocessing including tokenization. Notice from the data sample that two different columns contain text data -- `headline` and `short_description`. The more text data your Word2Vec model has, the better it will perform. Therefore, you'll want to combine the two columns before tokenizing each comment and training your Word2Vec model. 

In the cell below:

* Create a column called `'combined_text'` that consists of the data from the `'headline'` column plus a space character (`' '`) plus the data from the `'short_description'` column 
* Use the `'combined_text'` column's `.map()` method and pass in `word_tokenize`. Store the result returned in `data` 

In [3]:
df['combined_text'] = df["headline"] + " " + df["short_description"]
data = df["combined_text"].map(word_tokenize)

Inspect the first 5 items in `data` to see how everything looks. 

In [4]:
data[:5]

0    [There, Were, 2, Mass, Shootings, In, Texas, L...
1    [Will, Smith, Joins, Diplo, And, Nicky, Jam, F...
2    [Hugh, Grant, Marries, For, The, First, Time, ...
3    [Jim, Carrey, Blasts, 'Castrato, ', Adam, Schi...
4    [Julianna, Margulies, Uses, Donald, Trump, Poo...
Name: combined_text, dtype: object

Notice that although the words are tokenized, they are still in the same order they were in as headlines. This is important, because the words need to be in their original order for Word2Vec to establish the meaning of them. Remember that for a Word2Vec model you can specify a  **_Window Size_** that tells the model how many words to take into consideration at one time. 

If your window size was 5, then the model would start by looking at the words "Will Smith joins Diplo and", and then slide the window by one, so that it's looking at "Smith joins Diplo and Nicky", and so on, until it had completely processed the text example at index 1 above. By doing this for every piece of text in the entire dataset, the Word2Vec model learns excellent vector representations for each word in an **_Embedding Space_**, where the relationships between vectors capture semantic meaning (recall the vector that captures gender in the previous "king - man + woman = queen" example you saw).

Now that you've prepared the data, train your model and explore a bit!

## Training the Model

Start by instantiating a Word2Vec Model from `gensim`. 

In the cell below:

* Create a `Word2Vec` model and pass in the following arguments:
    * The dataset we'll be training on, `data`
    * The size of the word vectors to create, `size=100`
    * The window size, `window=5`
    * The minimum number of times a word needs to appear in order to be counted in  the model, `min_count=1` 
    * The number of threads to use during training, `workers=4`

In [5]:
model = Word2Vec(sentences=data, 
                 size=100, 
                 window=5, 
                 min_count=1, 
                 workers=4)

Now, that you've instantiated Word2Vec model, train it on your text data. 

In the cell below:

* Call the `.train()` method on your model and pass in the following parameters:
    * The dataset we'll be training on, `data`
    * The `total_examples`  of sentences in the dataset, which you can find in `model.corpus_count` 
    * The number of `epochs` you want to train for, which we'll set to `10`

In [6]:
model.train(sentences=data, 
            total_examples=model.corpus_count, 
            epochs=10)

(55565231, 67352790)

Great! You now have a fully trained model! The word vectors themselves are stored in the `Word2VecKeyedVectors` instance, which is stored in the `.wv` attribute. To simplify this, restore this object inside of the variable `wv` to save yourself some keystrokes down the line. 

In [7]:
wv = model.wv

## Examining Your Word Vectors

Now that you have a trained Word2Vec model, go ahead and explore the relationships between some of the words in the corpus! 

One cool thing you can use Word2Vec for is to get the most similar words to a given word. You can do this by passing in the word to `wv.most_similar()`. 

In the cell below, try getting the most similar word to `'Texas'`.

In [8]:
wv.most_similar("Texas")

[('Louisiana', 0.814021646976471),
 ('Ohio', 0.8125628232955933),
 ('Georgia', 0.805143415927887),
 ('Pennsylvania', 0.7991876006126404),
 ('Oregon', 0.7974849343299866),
 ('Illinois', 0.7926406860351562),
 ('Arkansas', 0.7877055406570435),
 ('Massachusetts', 0.7835580706596375),
 ('California', 0.781067967414856),
 ('Connecticut', 0.7778302431106567)]

Interesting! All of the most similar words are also states. 

You can also get the least similar vectors to a given word by passing in the word to the `.most_similar()` method's `negative` parameter. 

In the cell below, get the least similar words to `'Texas'`.

In [9]:
wv.most_similar(negative="Texas")

[('Coalesce', 0.4328553378582001),
 ('Hunger-Free', 0.4089893698692322),
 ('Per-Mile', 0.39148372411727905),
 ('Headstrong', 0.38952308893203735),
 ('Parent/Grandparent', 0.3865586221218109),
 ('Slammers', 0.38308271765708923),
 ('Animal-rescue', 0.3812732696533203),
 ('Adulterers', 0.365467369556427),
 ('Burned-Out', 0.365416020154953),
 ('autocrat', 0.3608374297618866)]

This seems like random noise. It is a result of the way Word2Vec is computing the similarity between word vectors in the embedding space. Although the word vectors closest to a given word vector are almost certainly going to have similar meaning or connotation with your given word, the word vectors that the model considers 'least similar' are just the word vectors that are farthest away, or have the lowest cosine similarity. It's important to understand that while the closest vectors in the embedding space will almost certainly share some level of semantic meaning with a given word, there is no guarantee that this relationship will hold at large distances. 

You can also get the vector for a given word by passing in the word as if you were passing in a key to a dictionary. 

In the cell below, get the word vector for `'Texas'`.

In [10]:
wv["Texas"]

array([-0.7913121 , -1.5907859 ,  2.057187  , -0.42884162, -1.6803608 ,
       -1.1729392 , -0.10327507, -1.3026866 ,  0.94784135,  0.11966374,
        0.77223593, -0.57000804,  1.6508424 , -0.56022775, -0.4431114 ,
        0.5242261 ,  1.4036169 , -2.1993146 ,  0.09791628, -0.9281352 ,
       -1.3657584 ,  1.4748478 , -0.4299901 , -0.20933348, -0.29051822,
       -1.5290221 , -1.2247442 , -0.506752  ,  0.09116069,  0.40119255,
       -1.1295111 ,  1.785658  , -1.1818693 , -0.17883179, -1.2756469 ,
        3.1721888 ,  0.49236053, -1.3636056 , -0.52372026, -0.3676548 ,
        1.1405543 ,  1.2007827 ,  2.1040516 , -1.1659591 ,  0.4190553 ,
        1.6981914 , -1.4868559 ,  0.02666454,  1.4475106 ,  0.4555815 ,
        0.8101924 , -0.7411209 ,  1.5748571 ,  1.9404981 ,  0.90290517,
       -0.93791425, -3.1257608 ,  1.0551726 , -1.1354018 ,  0.06015295,
        0.05791309,  0.85760874, -0.68391484, -1.9100705 , -2.3975582 ,
       -0.54347277, -0.06068506, -0.67821586,  0.881083  , -0.63

Now get all of the word vectors from the object at once. You can find these inside of `wv.vectors`. Try it out in the cell below.  

In [11]:
wv.vectors

array([[-0.4772642 ,  0.5237195 , -1.5613875 , ...,  1.412224  ,
        -0.21994391,  0.9939539 ],
       [-0.2224531 ,  0.66580135, -3.9561222 , ...,  1.913488  ,
         0.7317543 , -0.20741709],
       [ 1.2396152 ,  1.8259215 , -3.5676262 , ..., -0.26701498,
         1.802886  , -0.49603844],
       ...,
       [ 0.0470343 ,  0.0083683 ,  0.03152714, ...,  0.09067787,
        -0.11497129, -0.02388008],
       [-0.00815673,  0.0312268 , -0.01218776, ...,  0.03187821,
        -0.02141811,  0.03154721],
       [ 0.04423023, -0.01130465,  0.0395461 , ...,  0.09045943,
        -0.00875373,  0.05643199]], dtype=float32)

As a final exercise, try to recreate the _'king' - 'man' + 'woman' = 'queen'_ example previously mentioned. You can do this by using the `.most_similar()` method and translating the word analogies into an addition/subtraction formulation (as shown above). Pass the original comparison, which you are calculating a difference between, to the negative parameter, and the analogous starter you want to apply the same transformation to, to the `positive` parameter.

Do this now in the cell below. 

In [12]:
wv.most_similar(positive=['king', 'woman'], negative=["man"])

[('revival', 0.6176050901412964),
 ('classic', 0.6077556014060974),
 ('staple', 0.5935381650924683),
 ('centerpiece', 0.5901408195495605),
 ('jewel', 0.5831292867660522),
 ('princess', 0.5793606638908386),
 ('brunette', 0.5777530670166016),
 ('title', 0.5642304420471191),
 ('theme', 0.5630916953086853),
 ('reminiscent', 0.558822512626648)]

As you can see from the output above, your model isn't perfect, but 'Queen' and 'Princess' are still in the top 5. As you can see from the other word in top 5, 'reminiscent' -- your model is far from perfect. This is likely because you didn't have enough training data. That said, given the small amount of training data provided, the model still performs remarkably well! 

In the next lab, you'll reinvestigate transfer learning, loading in the weights from an open-sourced model that has already been trained for a very long time on a massive amount of data. Specifically, you'll work with the GloVe model from the Stanford NLP Group. There's not really any benefit from training the model ourselves, unless your text uses different, specialized vocabulary that isn't likely to be well represented inside an open-source model.

## Summary

In this lab, you learned how to train and use a Word2Vec model to create vectorized word embeddings!